In [ ]:
# TITLE

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

# magics and warnings
%load_ext autoreload
%autoreload 2
import warnings; warnings.simplefilter('ignore')

import os, random
import pandas as pd
import numpy as np

seed = 99
random.seed(seed)
np.random.seed(seed)

import nltk, sklearn
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")
sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 2.5})

In [5]:
# datasets

who_latest = "datasets/WHO_13_03_2020.csv"
dimensions_latest = "datasets/Dimensions_13_03_2020.csv"

df_who = pd.read_csv(who_latest)
df_dimensions = pd.read_csv(dimensions_latest)

In [6]:
df_who.head()

,Title,Authors,Abstract,Published Year,Published Month,Journal,Volume,Issue,Pages,Accession Number,DOI,Ref,Covidence #,Study,Notes,Tags
0,A case report of neonatal COVID-19 infection i...,NaN,"Abstract In December 2019, the 2019 novel coro...",NaN,NaN,Clinical Infectious Diseases,NaN,NaN,NaN,NaN,NaN,7885,#7931,NaN,NaN,* Case study/series; Epidemiology
1,Consensus of Chinese experts on protection of ...,NaN,Health professions preventing and controlling ...,2020.0,NaN,Chinese Journal of Dermatology,53,0,E001-E001,NaN,NaN,8458,#7986,NaN,NaN,* Opinion piece; Infection prevention and control
2,Immunization recommendations and safety and im...,NaN,NaN,2020.0,NaN,Chinese Journal of Pediatrics,58,0,E010-E010,NaN,NaN,8440,#7979,NaN,NaN,* Opinion piece; Infection prevention and control
3,Surveillance and Taxonomic Analysis of the Cor...,"Zhuang, Qingye; Liu, Shuo; Zhang, Xiaochun; Ji...",Coronaviruses (CoVs) are found in humans and a...,2020.0,NaN,Transboundary and emerging diseases,NaN,NaN,10.1111/tbed.13541-10.1111/tbed.13541,NaN,10.1111/tbed.13541,8255,#8319,Zhuang 2020,NaN,Other related diseases and viruses; Reservoir
4,Strategy of nursing care on the face skin inju...,"ZHOU, Qin; XUE, Jiao; MA, Ning Xia; TONG, Cui ...",For effective resistance to virus attack and i...,2020.0,NaN,Chinese Journal of Burns,36,0,E001-E001,NaN,NaN,8349,#8300,ZHOU 2020,NaN,* Opinion piece; Infection prevention and control


In [7]:
df_dimensions.head()

,Date added,Publication ID,DOI,PMID,PMCID,Title,Abstract,Source title,Source UID,Publisher,...,Research Organizations - standardized,GRID IDs,City of Research organization,Country of Research organization,Funder,UIDs of supporting grants,Times cited,Altmetric,Source Linkout,Dimensions URL
0,2020-03-11,pub.1125524902,10.21203/rs.3.rs-16657/v1,NaN,NaN,Estimating the daily trend in the size of the ...,NaN,Research Square,jour.1380788,Research Square,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://assets.researchsquare.com/files/rs-166...,https://app.dimensions.ai/details/publication/...
1,2020-03-11,pub.1125524903,10.21203/rs.3.rs-16702/v1,NaN,NaN,Early Chest CT Features of Patients with 2019 ...,NaN,Research Square,jour.1380788,Research Square,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.researchsquare.com/article/rs-1670...,https://app.dimensions.ai/details/publication/...
2,2020-03-11,pub.1125524733,10.21203/rs.3.rs-16094/v1,NaN,NaN,Multiple Negative of RT-PCR Testing of COVID-1...,NaN,Research Square,jour.1380788,Research Square,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.researchsquare.com/article/rs-1609...,https://app.dimensions.ai/details/publication/...
3,2020-03-11,pub.1125527453,10.3390/biology9030050,NaN,NaN,Understanding Unreported Cases in the COVID-19...,We develop a mathematical model to provide epi...,Biology,jour.1047179,MDPI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.mdpi.com/2079-7737/9/3/50/pdf,https://app.dimensions.ai/details/publication/...
4,2020-03-11,pub.1125525871,10.26434/chemrxiv.11955273,NaN,NaN,Exploring the Active Compounds of Traditional ...,<p><a></a><a></a><a></a><a><b>Objective</b></a...,ChemRxiv,jour.1315496,American Chemical Society (ACS),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,https://app.dimensions.ai/details/publication/...


In [19]:
# check DOIs
print("WHO")
print(df_who.shape)
print(df_who[pd.notna(df_who["DOI"])].shape)

print("Dimensions")
print(df_dimensions.shape)
print(df_dimensions[pd.notna(df_dimensions["DOI"])].shape)

WHO
(1754, 16)
(1425, 16)
Dimensions
(2571, 31)
(2509, 31)


In [30]:
df_join = df_dimensions.join(df_who.set_index("DOI"), how='inner', on="DOI", lsuffix='dimensions', rsuffix='who')

In [32]:
df_join = df_join[pd.notna(df_join["DOI"])]

In [33]:
df_join.shape

(689, 46)

In [34]:
df_join.head()

,Date added,Publication ID,DOI,PMID,PMCID,Titledimensions,Abstractdimensions,Source title,Source UID,Publisher,...,Journal,Volumewho,Issuewho,Pages,Accession Number,Ref,Covidence #,Study,Notes,Tags
3,2020-03-11,pub.1125527453,10.3390/biology9030050,NaN,NaN,Understanding Unreported Cases in the COVID-19...,We develop a mathematical model to provide epi...,Biology,jour.1047179,MDPI,...,Biology,9,3,NaN,NaN,5758,#5460,Liu 2020,NaN,* Narrative review; * Systematic review; Clini...
8,2020-03-11,pub.1125525062,10.22206/cysa.2020.v4i1.pp5-9,NaN,NaN,Nuevo coronavirus 2019-ncov: impacto en salud ...,Se trata de una revisión sistemática de una se...,Ciencia y Salud,jour.1301057,Instituto Tecnologico de Santo Domingo (INTEC),...,Ciencia y Salud,Vol. 4 Núm. 1 (2020):,NaN,NaN,NaN,5813,#5842,Pérez-Then 2020,NaN,Awaiting classification
18,2020-03-11,pub.1125510283,10.3760/cma.j.cn112137-20200229-00520,32145717.0,NaN,[Diagnosis and treatment of COVID-19: acute ki...,NaN,Chinese medical journal,jour.1297345,NaN,...,Zhonghua Yi Xue Za Zhi,100,0,E017-E017,32145717,6041,#5638,Yang 2020,NaN,Awaiting classification
21,2020-03-11,pub.1125510584,10.24171/j.phrp.2020.11.1.02,32149036.0,PMC7045880,Identification of Coronavirus Isolated from a ...,Objectives: Following reports of patients with...,Osong Public Health and Research Perspectives,jour.1045339,NaN,...,Osong Public Health and Research Perspectives,11,1,3-7,NaN,2205,#2631,Han 2020,NaN,"* Case study/series; Virology, immunology"
22,2020-03-11,pub.1125511458,10.3760/cma.j.cn501113-20200226-00070,32153170.0,NaN,[Clinical characteristics and influencing fact...,<b>Objective:</b> To understand the clinical c...,Chinese journal of hepatology,jour.1117115,NaN,...,Zhonghua Gan Zang Bing Za Zhi,28,0,E003-E003,32153170,6322,#5965,Yao 2020,NaN,"* Case study/series; Clinical aspects, diagnos..."


In [35]:
who_dois = df_who["DOI"].tolist()
dimensions_dois = df_dimensions["DOI"].tolist()

In [36]:
len(set(dimensions_dois).intersection(set(who_dois)))

654

#### TODO
*Clean up this mess and align the two (or more) datasets to have all publications.*

## Focus on Dimensions to test the Altmetrics API

Examples:
* PMID http://api.altmetric.com/v1/id/241939?key=
* DOI http://api.altmetric.com/v1/doi/10.1038/news.2011.490?key=

In [43]:
# get API key
import configparser
config = configparser.ConfigParser()
config.read("credentials/conf.ini")
api_key = config["ALTMETRICS"]["key"]

In [45]:
import requests

doi_base_url = "http://api.altmetric.com/v1/doi/"
pmid_base_url = "http://api.altmetric.com/v1/id/"

In [48]:
payload = {'key': api_key}
r = requests.get(doi_base_url+"10.3760/cma.j.cn501113-20200226-00070", params=payload)

In [49]:
r.json()

{'title': '[Clinical characteristics and influencing factors of patients with novel coronavirus pneumonia combined with liver injury in Shaanxi region].',
 'doi': '10.3760/cma.j.cn501113-20200226-00070',
 'pmid': '32153170',
 'altmetric_jid': '4f6fa9213cf058f61000d8e7',
 'issns': ['1007-3418'],
 'journal': 'Zhonghua gan zang bing za zhi Zhonghua ganzangbing zazhi Chinese journal of hepatology',
 'cohorts': {'pub': 1},
 'abstract': 'Objective: To understand the clinical characteristics, change of liver function, influencing factors and prognosis in hospitalized patients with coronavirus disease-19 (COVID-19) combined with liver injury. Methods: The general conditions, biochemical indicators of liver, blood clotting mechanism, routine blood test, UGT1A1 * 28 gene polymorphism and other data of 40 cases with COVID-19 admitted to the isolation ward of Tangdu Hospital were retrospectively analyzed. The clinical characteristics, influencing factors and prognosis of liver injury in patients w